<a href="https://colab.research.google.com/github/edwardoughton/satellite-image-analysis/blob/main/03_01_ggs416_26_02_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🛰️ GGS416 Satellite Image Analysis Week 3 🛰️

This week we will cover:
 * Downloading NAIP imagery
 * User-defined functions
 * Directory management
 * Cropping an image
 * Image segmentation

# Learning objectives

By the end of this week, students will be able to:

1. Download and manage NAIP satellite imagery using Python scripts and directory structures.
2. Write and execute user-defined Python functions to automate geospatial workflows.
3. Crop raster imagery to a region of interest using bounding boxes.
4. Implement threshold-based image segmentation to identify vegetation and impervious surfaces.
5. Apply spectral indices (ExG, NDWI) to segment environmental features.
6. Interpret segmentation results and evaluate their limitations.

# Approach to today's class content

We intend to do the following (with various exercises scattered inbetween):

* **Download** satellite imagery
* **Crop** the imagery to a region of interest
* **Inspect** the image to understand its visual and spectral properties
* **Segment** the image using threshold-based methods
* **Interpret** the segmentation results and evaluate their limitations

Every segmentation method follows a structure of (i) choosing a feature (brightness, spectral index, etc.), (ii) applying a basic threshold, and (iii) interpreting the results.

# Downloading NAIP imagery from Planetary Computer

Let us first install our packages, specify a bounding box, date and directory to export the image to.

If the directory does not exist, we will create it.

In [ ]:
# Example: Download NAIP image
#!pip -q install pystac-client planetary-computer requests

import os
import requests
from pystac_client import Client
import planetary_computer as pc

bbox = (-77.70, 38.50, -76.80, 39.10)          # min_lon, min_lat, max_lon, max_lat
datetime = "2022-01-01/2023-12-31"             # NAIP coverage on PC typically <= 2023
directory = "naip_downloads"
os.makedirs(directory, exist_ok=True)


Next, we will open our client catalog with Planetary Computer and search for our desired item based on the collection, bbox, etc.

Then we get our hypertext markup language (HTML) reference URL for the image (the `href`).

In [ ]:
# Example: Download NAIP image
#!pip -q install pystac-client planetary-computer requests

import os
import requests
from pystac_client import Client
import planetary_computer as pc

bbox = (-77.70, 38.50, -76.80, 39.10)          # min_lon, min_lat, max_lon, max_lat
datetime = "2022-01-01/2023-12-31"             # NAIP coverage on PC typically <= 2023
directory = "naip_downloads"
os.makedirs(directory, exist_ok=True)

catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1", modifier=pc.sign_inplace)
item = next(catalog.search(collections=["naip"], bbox=bbox, datetime=datetime, limit=1, method="POST").items())
href = pc.sign(item).assets["image"].href

print(href)

We can now download the image and save it to our folder.

In [ ]:
# Example: Download NAIP image
!pip -q install pystac-client planetary-computer requests

import os
import requests
from pystac_client import Client
import planetary_computer as pc

bbox = (-77.70, 38.50, -76.80, 39.10)          # min_lon, min_lat, max_lon, max_lat
datetime = "2022-01-01/2023-12-31"             # NAIP coverage on PC typically <= 2023
directory = "naip_downloads"
os.makedirs(directory, exist_ok=True)

catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1", modifier=pc.sign_inplace)
item = next(catalog.search(collections=["naip"], bbox=bbox, datetime=datetime, limit=1, method="POST").items())
href = pc.sign(item).assets["image"].href

filename = os.path.basename(href.split("?")[0])
out_path = os.path.join(directory, filename)

with requests.get(href, stream=True) as r:
  r.raise_for_status()
  with open(out_path, "wb") as f:
      for chunk in r.iter_content(1024 * 1024):
          if chunk:
              f.write(chunk)

print("Saved:", out_path)

In [ ]:
# Example: Download NAIP image
#!pip -q install pystac-client planetary-computer requests

import os
import requests
from pystac_client import Client
import planetary_computer as pc

bbox = (-77.70, 38.50, -76.80, 39.10)          # min_lon, min_lat, max_lon, max_lat
datetime = "2022-01-01/2023-12-31"             # NAIP coverage on PC typically <= 2023
directory = "naip_downloads"
os.makedirs(directory, exist_ok=True)

catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1", modifier=pc.sign_inplace)
item = next(catalog.search(collections=["naip"], bbox=bbox, datetime=datetime, limit=1, method="POST").items())
href = pc.sign(item).assets["image"].href

filename = os.path.basename(href.split("?")[0])
out_path = os.path.join(directory, filename)

if os.path.exists(out_path) and os.path.getsize(out_path) > 0:
    print("Already downloaded:", out_path)
else:
    with requests.get(href, stream=True) as r:
        r.raise_for_status()
        with open(out_path, "wb") as f:
            for chunk in r.iter_content(1024 * 1024):
                if chunk:
                    f.write(chunk)
    print("Saved:", out_path)


Now we want to plot the image to inspect it.

Remember: we need to re-order from (bands, rows, cols) in `rasterio`/`numpy`, to (rows, cols, bands) for matplotlib (hence, `transpose()`).

In [ ]:
# Example: Plot our image
import os
import rasterio
import matplotlib.pyplot as plt

directory = "naip_downloads"
path_in = os.path.join(directory, filename)
with rasterio.open(path_in) as ds:
    print("Bands:", ds.count)
    print("CRS:", ds.crs)
    print("Resolution:", ds.res)
    img = ds.read()  # (bands, rows, cols)

# Build RGB for plotting (scale to 0–1 for display)
# Raster values may be 0–255 or 0–65535; this normalizes safely.
rgb = img[:3].astype("float32")
rgb = rgb / (rgb.max() + 1e-6)
rgb = rgb.transpose(1, 2, 0)

plt.figure(figsize=(10, 10))
plt.imshow(rgb)
plt.title("NAIP RGB Preview of NOVA")
plt.axis("off")
plt.show()


## Defining a function

A key part of any programming language is being able to specify user-defined functions.

In python the convention is as follows:

* `def` starts the function definition (hence, `def`)
* then you can give it any name you want (e.g., `my_download_function()`)
* Next, we specify our function arguments in the parentheses, and finish with a colon (before going on to define what the function will do).
* Finally, we need to indent everything we want to be within the function. Convention is to finish the function using `return`, to be explicit about our output.

See the example below to understand this structure.

In [ ]:
# Example: Defining a basic function
def my_addition_function(num1, num2):
  output = num1 + num2
  return output

print(my_addition_function(2,2))
print(my_addition_function(4,4))
print(my_addition_function(10,10))


Now we can wrap our NAIP download script into a user function.

This looks complicated, but it is the same code we used before to download.

We are just putting the code inside the user-defined function structure we just covered.

In [ ]:
# Minimal NAIP download (Planetary Computer) -> saves 1 GeoTIFF locally
#!pip -q install pystac-client planetary-computer requests

import os
import requests
from pystac_client import Client
import planetary_computer as pc

def my_download_function(bbox, datetime, directory):

  os.makedirs(directory, exist_ok=True)

  catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1", modifier=pc.sign_inplace)
  item = next(catalog.search(collections=["naip"], bbox=bbox, datetime=datetime, limit=1, method="POST").items())
  href = pc.sign(item).assets["image"].href

  filename = os.path.basename(href.split("?")[0])
  out_path = os.path.join(directory, filename)

  if os.path.exists(out_path) and os.path.getsize(out_path) > 0:
      print("Already downloaded:", out_path)
  else:
      with requests.get(href, stream=True) as r:
          r.raise_for_status()
          with open(out_path, "wb") as f:
              for chunk in r.iter_content(1024 * 1024):
                  if chunk:
                      f.write(chunk)
      print("Saved:", out_path)

print("Here we placed the function in our working memory.")
print("We still need to execute the function to make it do anything.")

Now we can execute the function, by passing it a list of different bounding boxes.

This should download multiple NAIP images at once, swiftly automating this process.

In [ ]:
# Download many images at once for a single time period

bboxes = [
    (-77.70, 38.50, -76.80, 39.10), # NOVA
    (-82.60, 27.85, -82.30, 28.05), # Tampa Bay
    (-98.20, 37.70, -97.90, 37.90)  # Wichita, Kansas
]

for bbox in bboxes:
  my_download_function(bbox, datetime, directory)
  print(f"Completed: {bbox}")


## Exercise

Have a go at loading in last two NAIP images we downloaded and then visualizing in `matplotlib`.

You should be familiar with how to do this now given it is week 3, and having completed assignment 1.

To load the images, you will need to specify the correct filename and directory.


In [ ]:
# Place your plotting code here for image 2


In [ ]:
# Place your plotting code here for image 3


# Directory management

## Creating a folder (directory)

Python can create folders so your outputs stay organized. Being able to create and manage folders and files via scripting is a really important part of you being able to develop reproducible GIS workflows.

In [ ]:
# Example: Create a directory
import os

folder = "output"

os.makedirs(folder, exist_ok=True)
print("Folder ready:", folder)


Key idea:

* `os.makedirs` will create the directory/directories based on the path you provide.
* `exist_ok=True` prevents errors (and skips creation) if the folder already exists.
* This is how you make reproducible pipelines

## Creating a full file path (folder + filename)

Instead of manually writing your path string like so, `"output/myfile.tif"`, you should use `os.path.join()`.

In [ ]:
# Example: Dummy example for a filepath
import os

filename = "my_image.tif"
folder = "my_folder"
filepath = os.path.join(folder, filename)

print(filepath)

In [ ]:
# Example: This should be the filepath for one of our downloaded images
import os

filename = "m_3709824_nw_14_030_20230723_20231113.tif"
folder = "naip_downloads"
filepath = os.path.join(folder, filename)

print(filepath)

## Accessing parts of a path

It is very handy to be able to extract just the filename from a path, using the `os.path.basename()` function.

If needed, you can also just get the directory path using `os.path.dirname()`.

In [ ]:
# Example: Accessing parts of a path
import os

base = os.path.basename(filepath)
print(base)

In [ ]:
# Example: Get just the directory name
import os

directory = os.path.dirname(filepath)
print(directory)

## Checking if a file exists

We also want to be able to check if a file exists, e.g., to avoid re-downloading if it already does.

This speeds up scripts by avoiding redundant downloads.

In [ ]:
# Example: File should exist
import os

filename = "m_3709824_nw_14_030_20230723_20231113.tif"
folder = "naip_downloads"
filepath = os.path.join(folder, filename)

# Example: Check if a file exists
if os.path.exists(filepath):
    print("File exists!")
else:
    print("File not found.")

In [ ]:
# Example: File should not exist
import os

filename = "my_made_up_filename.tif"
folder = "naip_downloads"
filepath = os.path.join(folder, filename)

# Example: Check if a file exists
if os.path.exists(filepath):
    print("File exists!")
else:
    print("File not found.")

## List all files in a folder

Often, we might want to get all the files in a folder (e.g., when we want to do temporal change detection over multiple images).

To do so, we want to use the `os.listdir()` function.

In [ ]:
# Example: Let us use the sample data folder that comes with colab
import os

folder = "sample_data"
filenames = os.listdir(folder)
print(filenames) # Print the list
print(" ")

# Loop over all filenames
for filename in filenames:
  print(filename)

In [ ]:
# Example: Let us use our naip downloaded data
import os

folder = "naip_downloads"
filenames = os.listdir(folder)
print(filenames) # Print the list
print(" ")

# Loop over all filenames
for filename in filenames:
  print(filename)

## List only certain filetypes

We can also use some Python code to only get a certain type of file.

This is common when we only want to get `.tif` files.

Here, we can practise it on the sample data directory within colab.



In [ ]:
# Example: Get only one type of file from a directory
import os

folder = "sample_data"
filenames = os.listdir(folder)

# Loop over all filenames
for filename in filenames:
  if filename.endswith(".csv"): # Get only .csv files
    print(filename)

### Recap

To begin to work with multiple images over time you need to broadly understand:

* how folders organize data
* create folders using `os.makedirs()`
* build safe filepaths using `os.path.join()`
* extract parts of a path using `os.path.basename()` and `os.path.dirname()`
* using `os.path.exists()` to check file existence to prevent duplication
* list and find files using `os.listdir()`
* get only certain file types using Python `if` logic (e.g., and `endswith()`).

This is the foundation of reproducible GIS workflows, e.g., for satellite image analysis.

# Cropping an image

Cropping reduces file size and speeds up analysis, which is essential for rapid experimentation.

We will regularly need to crop images to have small examples which run quickly.

The bounding box below was developed manually from looking at the image in QGIS.

You can also develop them this way, or import your own geometry shapes to utilize.

Below we will load in our packages, define our directories, and open the image of interest.

In [ ]:
# Example: Cropping an image
import os
import rasterio
from rasterio.windows import from_bounds

in_path = os.path.join("naip_downloads", "m_3807708_sw_18_060_20231113_20240103.tif")
directory = "naip_cropped"
os.makedirs(directory, exist_ok=True)
out_path = os.path.join(directory, "crop_m_3807708_sw_18_060_20231113_20240103.tif")

with rasterio.open(in_path) as src:

  print("CRS:", src.crs)

  # min_lon, min_lat, max_lon, max_lat
  bbox = from_bounds(320285, 4307000, 321187, 4307769, src.transform)
  data = src.read(window=bbox)
  print(data)

The remaining two parts include:

* Updating the metadata, now we have changes the dimensions of the image.
* Writing both the image data and metadata out to our final output path.

In [ ]:
import os
import rasterio
from rasterio.windows import from_bounds

in_path = os.path.join("naip_downloads", "m_3807708_sw_18_060_20231113_20240103.tif")
directory = "naip_cropped"
os.makedirs(directory, exist_ok=True)
out_path = os.path.join(directory, "crop_m_3807708_sw_18_060_20231113_20240103.tif")

with rasterio.open(in_path) as src:

  print("CRS:", src.crs)

  # min_lon, min_lat, max_lon, max_lat
  bbox = from_bounds(320285, 4307000, 321187, 4307769, src.transform)
  data = src.read(window=bbox)

  # Update the raster metadata
  prof = src.profile
  prof.update(height=data.shape[1], width=data.shape[2],
              transform=rasterio.windows.transform(bbox, src.transform))

# Write to our file
with rasterio.open(out_path, "w", **prof) as dst:
    dst.write(data)

print("Wrote:", out_path)

Now we can view the image.

Remember, we need to re-organize from `rasterio`/`numpy` format (bands, rows, columns), to `matplotlib` format (rows, columns, bands).

In [ ]:
# View our cropped image
import matplotlib.pyplot as plt
import rasterio

with rasterio.open(out_path) as src:

  print(src.read([1,2,3]).shape)                        # rasterio format
  print(src.read([1,2,3]).transpose(1,2,0).shape)       # matplotlib format

  # Hence, we reorder the axes so they go: rows, columns, bands
  rgb = src.read([1,2,3]).transpose(1,2,0)

plt.imshow(rgb)
plt.axis("off")
plt.show()

# Image segmentation

Image segmentation groups together similar pixels which ideally represent real objects, e.g., we group pixels together for buildings, rivers, roads etc. Hence, this is about *image partitioning*.

This contrasts with image classification which is the next step (week 5 onwards in this class), e.g., what actually is this group of pixels?

Segmentation is a precursor to object-based analysis and GIS feature extraction.






## Threshold-based segmentation

This is the most basic approach to image segmentation, by merely creating a binary indicator to group similar pixels.

Key concepts:

* Binary masks
* Thresholding
* Spectral indices as segmentation tools
* Noise vs signal

## Example: Vegetation segmentation using an excess green threshold

This segments green vegetation using a simple color spectral index and then a `scikit-image` filter to threshold.

See the [scikit-image](https://scikit-image.org/) website and documentation for more information.

The specific documentation for the [threshold_otsu](https://scikit-image.org/docs/0.23.x/auto_examples/segmentation/plot_thresholding.html) function can also be viewed.

In [ ]:
# Example: Thresholding pixels with high green values
import os
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu

# Load NAIP image
in_path = os.path.join(
    "naip_cropped",
    "crop_m_3807708_sw_18_060_20231113_20240103.tif"
)

with rasterio.open(in_path) as src:
    img = src.read().astype("float32")

# Extract RGB bands
r, g, b = img[0], img[1], img[2]

# Get excess green index
exg = 2*g - r - b

# Automatic threshold using threshold_otsu()
thresh = threshold_otsu(exg)
veg_mask = exg > thresh

# Now plot
# Normalize RGB for display
rgb = img[:3] / (img[:3].max() + 1e-6)
rgb = np.transpose(rgb, (1, 2, 0))

# Overlay vegetation mask
overlay = rgb.copy()
overlay[veg_mask] = [0, 1, 0]  # green highlight

# Set up our panel plot
fig, ax = plt.subplots(1, 3, figsize=(18, 6))

ax[0].imshow(rgb)
ax[0].set_title("Original RGB")
ax[0].axis("off")

ax[1].imshow(exg, cmap="gray")
ax[1].set_title("Excess Green Index")
ax[1].axis("off")

ax[2].imshow(overlay)
ax[2].set_title("Vegetation Segmentation")
ax[2].axis("off")

plt.show()


### Exercise

1.   Critique the results, explaining what worked and what did not in this simple segmentation approach (2 minutes).
2.   Now have a go at changing the `veg_mask` threshold manually, e.g., `veg_mask = exg > 30` and compare the results.
3.   What could we do to improve this segmentation?





In [ ]:
# Enter your code here


## Example: Bright surface segmentation

Now we will target segmentation of bright impervious surfaces (roads/roofs) using a simple threshold value.


In [ ]:

import os
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu

# Load NAIP image
in_path = os.path.join(
    "naip_cropped",
    "crop_m_3807708_sw_18_060_20231113_20240103.tif"
)

with rasterio.open(in_path) as src:
    img = src.read().astype("float32")

# Extract RGB bands
r, g, b = img[0], img[1], img[2]

# Convert to grayscale brightness (simple arithmetic mean)
brightness = (r + g + b) / 3

# Threshold for bright areas
bright_thresh = threshold_otsu(brightness)
bright_mask = brightness > bright_thresh

# Now plot
# Normalize RGB for display
rgb = img[:3] / (img[:3].max() + 1e-6)
rgb = np.transpose(rgb, (1, 2, 0))

# Overlay brightness mask
overlay2 = rgb.copy()
overlay2[bright_mask] = [1, 0, 0]  # red highlight

# Set up our panel plot
fig, ax = plt.subplots(1, 3, figsize=(18, 6))

ax[0].imshow(rgb)
ax[0].set_title("Original RGB")
ax[0].axis("off")

ax[1].imshow(brightness, cmap="gray")
ax[1].set_title("Brightness")
ax[1].axis("off")

ax[2].imshow(overlay2)
ax[2].set_title("Bright Surface Segmentation")
ax[2].axis("off")

plt.show()


In [ ]:
# What about using 4 bands, e.g., the NIR band to help differentiate water?
import rasterio
import os

in_path = os.path.join(
    "naip_cropped",
    "crop_m_3807708_sw_18_060_20231113_20240103.tif"
)

with rasterio.open(in_path) as src:
    print("Band count:", src.count)


## Segmenting using the Normalized Difference Water Index (NDWI)

NDWI highlights water because this element absorbs NIR and reflects green light.

The equation can be defined as follows:

$$
NDWI = \frac{G - NIR}{G + NIR}
$$

where:

- \(G\) = Green band  
- \(NIR\) = Near Infrared band

In [ ]:
# Example: using the Normalized Difference Water Index (NDWI) to segment
import os
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu

# Load NAIP image
in_path = os.path.join(
    "naip_cropped",
    "crop_m_3807708_sw_18_060_20231113_20240103.tif"
)

with rasterio.open(in_path) as src:
    img = src.read().astype("float32")

# Extract RGB bands
r, g, b, nir = img[0], img[1], img[2], img[3]

# NDWI = (green - NIR) / (green + NIR)
# water body detection
ndwi = (g - nir) / (g + nir + 1e-6)

thresh = threshold_otsu(ndwi)
water_mask = ndwi > thresh

overlay = rgb.copy()
overlay[water_mask] = [0, 0.5, 1]

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.imshow(ndwi, cmap="gray")
plt.title("NDWI")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(overlay)
plt.title("Water Mask (NDWI)")
plt.axis("off")

plt.show()


### Exercise

1.   Critique the results, explaining what worked and what did not in this simple segmentation approach (2 minutes).
2.   Now have a go at changing the `thresh` threshold manually, e.g., `water_mask = ndwi > 0.7` and compare the results.
